## 보스턴 데이터셋의 특징 출력

In [3]:
!pip install scikit-learn==1.0.2

^C


scikit-learn 버전이 업그레이드 되면서 문제 발생    
특정 버전으로 다시 설치 필요 (윤리적 문제 이슈로 사용을 지양함)

In [1]:
import sklearn
print(sklearn.__version__)

1.3.0


In [ ]:
from sklearn.datasets import load_boston

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

dataset = load_boston()  # 데이터셋을 불러옴
# load_boston()
# 보스턴 집값 데이터를 딕셔너리 형태로 반환 
print(dataset.keys())  # 데이터셋의 키(요소들의 이름)을 출력

  Using cached scikit-learn-1.0.2.tar.gz (6.7 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'


  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [66 lines of output]
      Partial import of sklearn during the build process.
      setup.py:128: DeprecationWarning:
      
        `numpy.distutils` is deprecated since NumPy 1.23.0, as a result
        of the deprecation of `distutils` itself. It will be removed for
        Python >= 3.12. For older Python versions it will remain present.
        It is recommended to use `setuptools < 60.0` for those Python versions.
        For more details, see:
          https://numpy.org/devdocs/reference/distutils_status_migration.html
      
      
        from numpy.distutils.command.build_ext import build_ext  # noqa
      INFO: No module named 'numpy.distutils._msvccompiler' in numpy.distutils; trying from distutils
      Traceback (most recent call last):
        File "C:\Users\rnqhg\anaconda3\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_proces

- data : 우리가 사용할 특징 값
- target : 예측할 값. 정답이 있음
- feature_names : 각 특징의 이름
- DESCR : description의 약자로 데이터셋에 대한 전반적인 정보를 제공
- filename : 데이터셋의 csv 파일이 존재하는 위치

# 데이터의 구성요소 확인

In [ ]:
import pandas as pd

from sklearn.datasets import load_boston

dataset = load_boston()
dataFrame = pd.DataFrame(dataset["data"])
# 1. 데이터셋의 데이터 불러오기
dataFrame.columns = dataset["feature_names"]
# 2. 특정의 이름 불러오기
dataFrame["target"] = dataset["target"]
# 3. 데이터 프레임에 정답 추가

print(dataFrame.head())
# 4. 데이터프레임을 요약해서 출력
# head()
# 데이터프레임에서 일부 데이터만 출력합니다. 기본값은 5로, 첫번째 ~ 다섯번째 데이터를 출력합니다

# 선형회귀를 위한 MLP모델의 설계

**선형회귀**
- 데이터를 y와 x의 관계를 나타내는 직선으로 나타내는 방법
- 회귀에 의해 얻은 결과를 실제 데이터와 비교해 오차를 줄여나가는 방식으로 학습
- 오차의 제곱에 대한 평균을 취하는 평균 제곱 오차(MSE)

In [ ]:
import torch
import torch.nn as nn

from torch.optim.adam import Adam

# 1. 모델 정의
# torch.nn.Sequential() 객체에 모듈을 집어넣어주면, 파이토치가 알아서 계산
# 선형 회귀 모델이므로 파이토치의 nn.Linear 모듈을 이용
# MLP층 : 뉴런이 다음 층의 모든 뉴런과 연결되어 있기 때문에 완전연결층이라고 부름

# 선형회귀 MLP 모델의 순서
# (1). 먼저 입력층에 입력데이터가 들어옵니다.
# (2). 은닉층에 전달된 입력 데이터의 특징으로부터 정보를 추출합니다.
# (3). 출력층의 예측값과 실제 정답을 비교해 손실을 꼐산합니다.
# (4). 손실을 계산했으면 가중치를 수정하기 위해 오차를 역전파 합니다.

model = nn.Sequential(
    nn.Linear(13, 100),  # 13개의 특징을 받아 100개의 특징을 반환
    nn.ReLU(),
    nn.Linear(100, 1)
)

X = dataFrame.iloc[:, :13].values  # 2. 정답을 제외한 특징을 X에 입력
Y = dataFrame["target"].values  # 데이터 프레임의 target의 값을 추출
# 데이터 13개를 X에 입력하고 싶기 때문에 지금은 문자열 13개를 넣어줘야 합니다.
# 특징들의 위치로 값을 불러올 수 있으면 편하기 때문에 iloc를 이용해 이름이 아닌 위치로 호출할 수 있게 해준 뒤, 정답을 제외한 모든 데이터를 불러옵니다.

batch_size = 100
learning_rate = 0.001


# 3. 가중치를 수정하기 위한 최적화 정의
optim = Adam(model.parameters(), lr=learning_rate)
# Adam은 가장 많이 쓰이는 최적화 기법입니다.
# 최적화 기법은 역전파된 오차를 이용해 가중치를 수정하는 기법을 말합니다.
# Adam()은 최적화가 필요한 모델의 가중치들과 학습률을 입력으로 받습니다.

# 에포크 반복
for epoch in range(200):

    # 배치 반복
    for i in range(len(X)//batch_size):
        start = i*batch_size  # 4. 배치 크기에 맞게 인덱스를 지정
        end = start + batch_size

        # 파이토치 실수형 텐서로 변환
        x = torch.FloatTensor(X[start:end])
        y = torch.FloatTensor(Y[start:end])
        # 전체 데이터 크기를 배치 크기로 나누고 나서 시작 지점과 끝나는 지점을 계산합니다.
        # FloatTensor()를 사용해서 자료형을 실수형으로 변환해줍니다.
        # FloatTensor(A) 
        # 설명 : 객체 A를 실숫값을 갖는 파이토치 텐서로 변환해줍니다.

        optim.zero_grad()  # 5. 가중치의 기울기를 0으로 초기화
        # 최적화를 실행하기 전 모든 기울기를 0으로 초기화 합니다.
        # 이전 배치에서 계산된 기울기가 남아 있기 때문에 배치마다 초기화 해줍니다.
        preds = model(x)  # 6. 모델의 예측값 계산
        loss = nn.MSELoss()(preds, y)  # 7. MSE 손실 계산
        # 실젯값과 예측값의 차이를 제곱하고, 모든 입력에 대해 평균을 내는 오차를 말합니다.
        # loss.backward()는 오차를 역전파시킬 기울기를 저장합니다.
        # optim.step()은 최적화 함수에 맞춰 오차 역전파를 수행합니다.
        loss.backward()
        optim.step()


    if epoch % 20 == 0:
        print(f"epoch{epoch} loss:{loss.item()}")


# MSELoss()(preds, target) 
# 설명 : preds와 target에 대한 제곱 평균 오차를 구하는 함수입니다.
# Adam(params, lr)
# 설명 : 학습률 lr을 갖고, 모델의 가중치에 대해 Adam 최적화를 해주는 객체입니다.

# 모델 성능 평가

In [ ]:
prediction = model(torch.FloatTensor(X[0, :13]))
real = Y[0]
print(f"prediction:{prediction.item()} real:{real}")